In [6]:
import shap
import numpy as np
import pandas as pd
from datetime import datetime
from functools import reduce
from numpy import nan
import seaborn as sns
import  math
import sys
import os
import datetime
import IPython 

import sklearn
from sklearn import preprocessing
import time
import matplotlib.pyplot as plt

# to make this notebook's output stable across runs
np.random.seed(42)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE,SMOTENC
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import roc_auc_score, auc, precision_recall_curve

import xgboost as xgb
import xgboost
from xgboost import XGBClassifier

from numpy import mean
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble  import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn import svm
# from bentoml import env, artifacts, api, BentoService
# from bentoml.adapters import JsonInput, JsonOutput, DataframeInput, DataframeOutput
# from bentoml.frameworks.sklearn import SklearnModelArtifact
# from bentoml.service.artifacts.common import  PickleArtifact

import warnings
warnings.filterwarnings("ignore")

In [9]:
# @env(infer_pip_packages=True)
class Shap():
    def shap_imp(self, df):
        final_data = df.copy()

        #print(final_data)

        final_data = final_data.set_index(['patient_gid'])

        ## Dataset split

        X = final_data.loc[:, final_data.columns != 'conversion_flag']
        y = final_data.loc[:, final_data.columns == 'conversion_flag']


        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
#         print("X_train",X_train)
#         print("y_train",y_train)
        # define dataset

        col_list = X_train.columns
        X = X_train
        y = y_train

        ## LOGISTIC
        lr = LogisticRegression(class_weight='balanced')
        lr_clf = lr.fit(X_train, y_train)

        ### SHAP

        coef_importances = pd.Series(lr.coef_[0], index=X_train.columns)

        # sort by absolute value, highest to lowest
        sorted_by_abs_val = coef_importances.abs().sort_values(ascending=False).index
        coef_importances = coef_importances[sorted_by_abs_val]

        pred_lr = pd.Series(lr.predict_proba(X_test)[:, 1],
                            index=X_test.index)

        #%%time
        background_data = X_train.sample(X.shape[0], random_state=3)
        background_means = shap.kmeans(background_data, k=2)

        # SHAP
        predictor_function_logreg = lambda x: lr.predict_proba(x)[:, 1]    
        explainer = shap.KernelExplainer(predictor_function_logreg, background_means, link='logit')

        # take a sample of the dataset, n=1000
        selected_indices_logreg = pred_lr.sample(n=X_test.shape[0], random_state=1).index
        selected_examples_logreg = X_test.loc[selected_indices_logreg, :]
        
        ##shap importance 
        shap_values_logreg = explainer.shap_values(selected_examples_logreg, l1_reg='num_features(15)',matplotlib=True,silent=True)
        mean_shap_importances = (pd.DataFrame(shap_values_logreg, columns=selected_examples_logreg.columns).abs().mean().sort_values(ascending=False))
        shap_imp = pd.DataFrame(mean_shap_importances)
        shap_imp.reset_index(inplace = True)
        
        ## dataframe with shap importance values
        shap_imp.columns =['grain', 'SHAP_Importance']
        normalized_sum = sum(shap_imp["SHAP_Importance"])
        shap_imp["SHAP_Norm_Imp"] = shap_imp["SHAP_Importance"] / normalized_sum

        print(shap_imp)

        #return shap_imp

#     @api(input=DataframeInput(orient="records"),batch=True,output=DataframeOutput(output_orient='records'))
    def predict(self, df):
        
        shap_imp = self.shap_imp(df)
        #shap_imp = shap_imp.reset_index()
#         df_final=pd.DataFrame()
#         ## model output is returned in a concated form
#         df_final['all_columns'] =(shap_imp['grain'].astype(str)+'|'+shap_imp['SHAP_Importance'].astype(str)+'|'+shap_imp['SHAP_Norm_Imp'].astype(str))
#         list_zero=['0']*(df.shape[0] - shap_imp.shape[0])
#         xtra = {'all_columns': list_zero}
#         df_final = df_final.append(pd.DataFrame(xtra))
#         return df_final

In [10]:
shapley = Shap()
df = pd.read_csv("./input/NC_SHAP_sample.csv")
output = shapley.predict(df)

  grain  SHAP_Importance  SHAP_Norm_Imp
0    P4         0.359233       0.511108
1    P2         0.213330       0.303521
2    P1         0.130288       0.185371
3    P3         0.000000       0.000000
